In [1]:
!pip install pydantic-ai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.0/765.0 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.1.2
    Uninstalling tenacity-9.1.2:
      Successfully uninstalled tenacity-9.1.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.0
    Uninstalling huggingface-hub-0.33.0:
      Successfully uninstalled huggingface-hub-0.33.0
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.20.0
    Uninstalling google-genai-1.20.0:
      Successfully uninstalled google-genai-1.20.0

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from pydantic_ai import Agent, RunContext


In [4]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
<QUESTION>
{question}
</QUESTION>

""".strip()
chat_agent = Agent(  
    'google-gla:gemini-2.0-flash'
)

In [5]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [6]:
from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [7]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [8]:
from typing import Dict


@chat_agent.tool
def search_tool(ctx: RunContext, query: str) -> Dict[str, str]:
    """
    Search the FAQ for relevant entries matching the query.

    Parameters
    ----------
    query : str
        The search query string provided by the user.

    Returns
    -------
    list
        A list of search results (up to 5), each containing relevance information 
        and associated output IDs.
    """
    print(f"search('{query}')")
    return search(query)

In [20]:
user_prompt = "i have trouble installing docker"
#developer_prompt.format(question=user_prompt)
agent_run = await chat_agent.run(user_prompt)
print(agent_run.output)

search('installing docker')
I'm sorry to hear you're having trouble installing Docker. I can help you troubleshoot this issue. To assist you better, could you please provide more information about the problem you're encountering? For example:

*   What operating system are you using?
*   What steps have you taken so far to install Docker?
*   What specific error messages are you receiving?

In the meantime, here are some general troubleshooting steps that might help:

1.  **Check the Docker documentation:** The official Docker documentation provides detailed installation instructions for various operating systems. Make sure you're following the instructions for your specific OS.
2.  **Verify system requirements:** Ensure that your system meets the minimum requirements for running Docker.
3.  **Check for conflicting software:** Some software might conflict with Docker. Disable or uninstall any potentially conflicting software and try installing Docker again.
4.  **Run the installer as a